In [90]:
import matplotlib.pyplot as plt
import numpy as np
from pycocotools.coco import COCO
import json
from pycocotools.cocoeval import COCOeval
import pickle

pickleFilePath = '../data/data/data/da_D0.pkl'
pickleFilePath = '../data/data/data/ub_D0.pkl'
pickleFile = open(pickleFilePath,'rb')
pred_pkl = pickle.load(pickleFile)

anno_path = '../data/data/bdd100k/annotations/det_val_D0.json'
save_path = '../data/experiment/result.json'
coco = COCO(anno_path)

def get_pred_data_pkl(img_id):
    index = img_id - 1
    cls_score = pred_pkl[index]['pred_instances']['scores'].numpy()
    bbox = pred_pkl[index]['pred_instances']['bboxes'].numpy()
    cat_ids = pred_pkl[index]['pred_instances']['labels'].numpy()
    return cat_ids,cls_score,bbox

def get_gt_data(img_id):
    anno_id = coco.getAnnIds(img_id)
    anno_data = coco.loadAnns(anno_id)
    bbox_list = []; cate_list = []
    for anno in anno_data:
        x1, y1, w, h = anno['bbox']
        bbox_list.append([x1, y1, x1+w, y1+h])
        cate_list.append(anno['category_id'])
    bbox_data = np.array(bbox_list)
    cate_data = np.array(cate_list)
    return cate_data,bbox_data

loading annotations into memory...
Done (t=0.36s)
creating index...
index created!


In [10]:
def calculate_iou(box1, box2):
    # Calculate Intersection over Union (IoU) between two bounding boxes
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area
    return iou

def calculate_ap(recall, precision):
    # Calculate Average Precision (AP) given recall and precision arrays
    recall = np.concatenate(([0.], recall, [1.]))
    precision = np.concatenate(([0.], precision, [0.]))

    for i in range(len(precision) - 1, 0, -1):
        precision[i - 1] = max(precision[i - 1], precision[i])

    indices = np.where(recall[1:] != recall[:-1])[0]
    ap = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])
    return ap

def calculate_map(img_id,iou_threshold=0.5):
    cat_id_gt, bbox_gt = get_gt_data(img_id)
    cat_id_pred,scores_pred,bbox__pred = get_pred_data_pkl(img_id)
    average_precisions = []
    for index in range(10):
        cid = index + 1
        index_class_gt = np.where(cat_id_gt == cid)[0]
        index_class_pred = np.where(cat_id_pred == cid)[0]
        num_pred = index_class_pred.shape[0]
        num_gt = index_class_gt.shape[0]

        if num_pred == 0:
            average_precisions.append(0)
            continue

        class_pred_score = scores_pred[index_class_pred]
        class_pred_bbox = bbox__pred[index_class_pred]
        class_pred_cid = cat_id_pred[index_class_pred]
        class_gt_bbox = bbox_gt[index_class_gt]

        class_sorted_indices = np.argsort(-class_pred_score)
        class_pred_score_sorted = class_pred_score[class_sorted_indices]
        class_pred_bbox_sorted = class_pred_bbox[class_sorted_indices]
        class_pred_cid_sorted = class_pred_cid[class_sorted_indices]

        true_positives = np.zeros(num_pred)
        false_positives = np.zeros(num_pred)
        matched_ground_truth = set()

        for index_in_class in range(num_pred):
            if num_gt == 0:
                false_positives[index_in_class] = 1
                continue

            ious = [calculate_iou(class_pred_bbox_sorted[index_in_class], box) for box in class_gt_bbox]
            max_iou = max(ious)
            max_iou_index = np.argmax(ious)

            if max_iou >= iou_threshold and max_iou_index not in matched_ground_truth:
                true_positives[index_in_class] = 1
                matched_ground_truth.add(max_iou_index)
            else:
                false_positives[index_in_class] = 1

        true_positives = np.cumsum(true_positives)
        false_positives = np.cumsum(false_positives)
        recall = true_positives / num_gt
        precision = true_positives / (true_positives + false_positives)

        ap = calculate_ap(recall, precision)
        average_precisions.append(ap)

    mAP = np.mean(average_precisions)
    return mAP


#     # Calculate mean Average Precision (mAP) for object detection
#     num_classes = len(predictions)
#     average_precisions = []

#     for class_id in range(num_classes):
#         class_ground_truth = ground_truth[class_id]
#         class_predictions = predictions[class_id]

#         num_ground_truth = len(class_ground_truth)
#         num_predictions = len(class_predictions)

#         if num_predictions == 0:
#             average_precisions.append(0)
#             continue


#         sorted_indices = np.argsort(-class_predictions[:, 4])
#         class_predictions = class_predictions[sorted_indices]

#         true_positives = np.zeros(num_predictions)
#         false_positives = np.zeros(num_predictions)
#         matched_ground_truth = set()

#         for i in range(num_predictions):
#             prediction = class_predictions[i]
#             if num_ground_truth == 0:
#                 false_positives[i] = 1
#                 continue

#             ious = [calculate_iou(prediction[:4], box) for box in class_ground_truth]
#             max_iou = max(ious)
#             max_iou_index = np.argmax(ious)

#             if max_iou >= iou_threshold and max_iou_index not in matched_ground_truth:
#                 true_positives[i] = 1
#                 matched_ground_truth.add(max_iou_index)
#             else:
#                 false_positives[i] = 1

#         true_positives = np.cumsum(true_positives)
#         false_positives = np.cumsum(false_positives)
#         recall = true_positives / num_ground_truth
#         precision = true_positives / (true_positives + false_positives)

#         ap = calculate_ap(recall, precision)
#         average_precisions.append(ap)

#     mAP = np.mean(average_precisions)
#     return mAP

In [ ]:
# 准备数据
ground_truth_images = []  # 存储每张图片的 ground truth，可以是一个列表，每个元素表示一张图片的 ground truth
predictions_images = []  # 存储每张图片的 predictions，可以是一个列表，每个元素表示一张图片的 predictions

# 循环处理每张图片
average_precisions = []
for ground_truth, predictions in zip(ground_truth_images, predictions_images):
    ap = calculate_map(ground_truth, predictions)
    average_precisions.append(ap)

# 计算 mAP
mAP = np.mean(average_precisions)
print("mAP:", mAP)

In [99]:
coco_result = []
for sample_img in pred_pkl:
    result = []
    img_id = int(sample_img['img_id'])
    ann_num = len(sample_img['pred_instances']['labels'])
    for index in range(ann_num):
        label = int(sample_img['pred_instances']['labels'][index])

        label_map = [1,2,3,5,4,8,7,9,10,6]
                   #[1,2,3,4,5,6,7,8, 9,10]
        label = label_map[label-1]


# {"categories": [{"supercategory": "none", "id": 1, "name": "person"}, {"supercategory": "none", "id": 2, "name": "rider"}, {"supercategory": "none", "id": 3, "name": "car"}, {"supercategory": "none", "id": 4, "name": "bus"}, {"supercategory": "none", "id": 5, "name": "truck"}, {"supercategory": "none", "id": 6, "name": "bike"}, {"supercategory": "none", "id": 7, "name": "motor"}, {"supercategory": "none", "id": 8, "name": "traffic light"}, {"supercategory": "none", "id": 9, "name": "traffic sign"}, {"supercategory": "none", "id": 10, "name": "train"}]


        score = float(sample_img['pred_instances']['scores'][index])
        bbox_xyxy = sample_img['pred_instances']['bboxes'][index].numpy().tolist()
        if score < 0.04:
            continue
        xs = bbox_xyxy[0]
        ys = bbox_xyxy[1]
        ws = bbox_xyxy[2] - xs
        hs = bbox_xyxy[3] - ys
        bbox_xywh = [float(xs), float(ys), float(ws), float(hs)]   
        result += [
            {
                'image_id': img_id,
                'category_id': label,
                'bbox': bbox_xywh,
                'score': score
            }
        ]
    # result = sorted(result, key=lambda x: x['score'])[-100:]
    coco_result += result

coco_dt = coco.loadRes(coco_result)
coco_eval = COCOeval(coco, coco_dt,'bbox')
# coco_eval.params.iouThrs = 0.05
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

Loading and preparing results...
DONE (t=0.49s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=9.06s).
Accumulating evaluation results...
DONE (t=1.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.049
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.066
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.105
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.047
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.078
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.084
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10

In [100]:
img = 200
ann = coco_dt.getAnnIds(img)
anndata = coco_dt.loadAnns(ann)

ann_gt = coco.getAnnIds(img)
anndata_gt = coco.loadAnns(ann_gt)


In [101]:
xx1 = []
xx2 = []
for instance in anndata:
    xx1.append(instance['category_id'])
for instance in anndata_gt:
    xx2.append(instance['category_id'])
print(xx1)
print(xx2)

[1, 1, 1, 5, 5, 1, 1, 1, 5, 5, 5, 2, 3, 1, 1, 1, 1, 5, 5, 1, 5, 1, 1, 1, 5, 1, 1, 1, 1, 5, 1, 1, 3, 1, 1, 1, 1, 5, 5, 1, 1, 5, 1, 5, 5, 5, 1, 1, 1, 1, 1, 3, 5, 1, 3, 2, 1, 1, 3, 5, 5, 1, 5, 3, 5]
[8, 8, 8, 8, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9]


In [102]:
from PIL import Image, ImageDraw
# 定义画布尺寸
canvas_width = 1280
canvas_height = 720

# 创建空白画布
canvas = Image.new('RGB', (canvas_width, canvas_height), 'white')

# 创建绘图对象
draw = ImageDraw.Draw(canvas)
for instance in anndata_gt:
    bbox = instance['bbox']
    print(instance['category_id'])
    colors = [
    (255, 0, 0),      # 红色
    (0, 255, 0),      # 绿色
    (0, 0, 255),      # 蓝色
    (255, 255, 0),    # 黄色
    (255, 0, 255),    # 品红色
    (0, 255, 255),    # 青色
    (128, 0, 0),      # 深红色
    (0, 128, 0),      # 深绿色
    (0, 0, 128),      # 深蓝色
    (128, 128, 128)   # 灰色
    ]
    # 定义边界框数据
    x = bbox[0]
    y = bbox[1]
    w = bbox[2]
    h = bbox[3]
    # 绘制边界框
    draw.rectangle([(x, y), (x+w, y+h)], outline=colors[instance['category_id']-1], width=2)
# 保存画布
canvas.save('./img/output_gt.png')

print('------------------')

canvas1 = Image.new('RGB', (canvas_width, canvas_height), 'white')
draw1 = ImageDraw.Draw(canvas1)
for instance in anndata:
    if instance['score'] < 0.10:
        continue
    bbox = instance['bbox']
    print(instance['category_id'])
    colors = [
    (255, 0, 0),      # 红色
    (0, 255, 0),      # 绿色
    (0, 0, 255),      # 蓝色
    (255, 255, 0),    # 黄色
    (255, 0, 255),    # 品红色
    (0, 255, 255),    # 青色
    (128, 0, 0),      # 深红色
    (0, 128, 0),      # 深绿色
    (0, 0, 128),      # 深蓝色
    (128, 128, 128)   # 灰色
    ]
    # 定义边界框数据
    x = bbox[0]
    y = bbox[1]
    w = bbox[2]
    h = bbox[3]
    # 绘制边界框
    draw1.rectangle([(x, y), (x+w, y+h)], outline=colors[instance['category_id']-1], width=2)
# 保存画布
canvas1.save('./img/output_pred.png')

8
8
8
8
9
9
9
3
3
3
3
3
3
3
9
9
9
------------------
1
1
1
5
5
1
1
1
5
5
5
2
3
1
1
1
1
5
5
1
5
